In [0]:
df = spark.read.format('csv')\
    .option('Inferschema','True')\
    .option('header','True')\
        .load('/Volumes/workspace/f1_bronze_layer/f1_case_study/races.csv')    

In [0]:
display(df)

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType

my_schema = StructType(
  fields =  [
       (StructField('raceId', IntegerType(), True)),
       (StructField('year', IntegerType(), True)),
       (StructField('round', IntegerType(), True)),
       (StructField('circuitId', IntegerType(), True)),
       (StructField('name', StringType(), True)),
       (StructField('date', DateType(), True)),
       (StructField('time', StringType(), True)),
       (StructField('url', StringType(), True))
    ]
)

In [0]:
df = spark.read.format('csv')\
    .schema(my_schema)\
    .option('header','True')\
        .load('/Volumes/workspace/f1_bronze_layer/f1_case_study/races.csv')    


In [0]:
display(df)

In [0]:
from pyspark.sql.functions import current_timestamp, to_timestamp,concat,lit,col,when

races_timestamp_df = df.withColumn('ingestion_date',current_timestamp())\
    .withColumn('race_timestamp',to_timestamp(concat(col('date'),lit(' '),when(col('time') == '\\N', None).otherwise(col('time'))),'yyyy-MM-dd HH:mm:ss'))

In [0]:
display(races_timestamp_df)

In [0]:
races_timestamp_df.write \
    .partitionBy('year') \
    .format('parquet') \
    .mode('overwrite') \
    .save('/Volumes/workspace/f1_bronze_layer/f1_case_study/races_timestamp')
